In [1]:
import random

In [2]:
class BraidSignGroup:
    def __init__(self, n, p, l):
        self.p = p
        self.l = l
        self.n = n
        self.B = BraidGroup(n)
        self.identityElem = self.B([])
        self.gens = self.B.generators()
        self.leftGens = [g for g in self.gens if self.gens.index(g) + 1 <= (int(n / 2) - 1)]
        self.rightGens = [g for g in self.gens if self.gens.index(g) + 1 >= (int(n / 2) + 1)]
        self.magic = 5


    def hash(self, U, m, T):
        parts = [U]
        parts.append(int("".join([str(part) for part in [elem.encode("utf-8").hex() for elem in m]]), 16))
        #parts.append(int("".join([str(part) for part in [elem.encode("hex") for elem in m]]), 16))
        parts.append(self.getIntFromBraid(T))
        return int("".join([str(part) for part in parts])) % (self.p-1) + 1
    
    def hash1(self, m):
        return self.B([int(d) % (self.B.strands() - 1) + 1 for d in str(abs(hash(m)))])

    def getPowForLeftForm(self, leftForm):
        deltaPower = leftForm[0]
        if deltaPower == self.identityElem:
            return 0
        else:
            return deltaPower.exponent_sum()/self.B.delta().exponent_sum()


    def getIntFromBraid(self, braid):
        leftForm = braid.left_normal_form()
        parts = [self.getPowForLeftForm(leftForm)]
        for i in range(1, len(leftForm)):
            parts.append(int("".join([str(digit) for digit in leftForm[i].permutation()])))
        return int("".join([str(part) for part in parts]))


    def ifFromSubgroupL(self, braid):
        leftForm = braid.left_normal_form()
        inf = self.getPowForLeftForm(leftForm)
        sup = inf + len(leftForm) - 1
        if(0 <= inf and inf <= sup and sup <= self.l):
            return True
        return False

    def randLBElement(self):
        r = random.randint(1, self.magic)
        e = self.identityElem
        for i in range(0, r):
            e *= random.choice(self.leftGens)
        return e

    def randRBElement(self):
        r = random.randint(1, self.magic)
        e = self.identityElem
        for i in range(0, r):
            e *= random.choice(self.rightGens)
        return e
    def randLBlElement(self):
        while True:
            elem = self.randLBElement()
            if(self.ifFromSubgroupL(elem)):
                return elem


    def randRBlElement(self):
        while True:
            elem = self.randRBElement()
            if(self.ifFromSubgroupL(elem)):
                return elem

In [119]:
bsg = BraidSignGroup(24, 17, 16)

In [121]:
class P:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
    def step_1(self):
        a = bsg.randLBElement()
        b = self.z * a / self.z
        self.a = a
        self.b = b
        return a, b
    
    def step_3(self, r):
        c = self.z * self.x**(r) * self.a / self.z
        return c

In [123]:
class P_crack:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
    def step_1(self):
        a = bsg.randLBElement()
        b = self.z * a / self.z
        self.a = a
        self.b = b
        return a, b
    
    def step_3(self, r):
        c = self.y**r * self.a
        return c

In [124]:
class V:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def step_2(self, a, b):
        self.a = a
        self.b = b
        r = random.randint(-15, 16)
        self.r = r
        return r
    
    def step_4(self, c):
        l = (c / self.b).is_conjugated(self.x**self.r)
        r = (self.y**(-self.r)*c).is_conjugated(self.a)   
        return l and r

In [76]:
p = P(x, y, z)
v = V(x, y)

a, b = p.step_1()
r = v.step_2(a, b)
c = p.step_3(r)
v.step_4(c)

True True


True

In [94]:
#False alarm
a, b, r, c

(s0, s0*s2*s0*s2^-1*s0^-1, -880, s2*s1^-880*s2^-1*s0)